In [289]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [290]:
df_warehouse=pd.read_excel(r"C:\Users\Ratan Kumar Jha\Desktop\new\new_Warehouse Asset Module Check sheet Sample Master Sheet.xlsx")

In [291]:
df_warehouse.to_csv("warehouse_data.csv",index=False)

In [292]:
warehouse_Data = pd.read_csv("C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_data.csv")

In [293]:
assest_initiation_invent_sb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\new\ASSET_INITIATION_INVENTORY_SB.CSV")

In [294]:
assest_initiation_invent_nb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\new\ASSET_INITIATION_INVENTORY_NB.CSV")

In [295]:
assest_initiation_invent_sb.count()

MeterNumber                                  991462
MFG_SERIAL_NUM                               991462
UDC_ID                                       985957
NETWORK_ID                                   985957
InstallationNumber                           957400
Status                                       991462
INSERT_TIME                                  991462
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    991462
dtype: int64

# change data type into string

In [296]:
# Convert all columns to string data type

warehouse_Data = warehouse_Data.astype(str)
assest_initiation_invent_nb = assest_initiation_invent_nb.astype(str)
assest_initiation_invent_sb = assest_initiation_invent_sb.astype(str)


# change column name into upper string

In [297]:
warehouse_Data.columns = warehouse_Data.columns.str.upper()
assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.upper()
assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.upper()

# trim the trailling and leading spaces of needed column

In [298]:
# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_Data = warehouse_Data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
assest_initiation_invent_nb = assest_initiation_invent_nb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
assest_initiation_invent_sb = assest_initiation_invent_sb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)


# strip the spaces in column 

In [299]:




# strip the spaces in column 

warehouse_Data.columns = warehouse_Data.columns.str.strip()
assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.strip()
assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.strip()



In [300]:
warehouse_Data.count()

SL. NO.                    5537
METER NO.                  5537
PHASE TYPE                 5537
FINAL HES MAPPING          5537
CURRENT LOCATION           5537
TOTAL METERS STATUS        5537
SUPPLIER                   5537
FINAL CATEGORY OK          5537
IME AS PER L&T SHIPMENT    5537
SIM 19 DISIT               5537
IMSI                       5537
IP                         5537
dtype: int64

In [301]:
warehouse_Data.columns

Index(['SL. NO.', 'METER NO.', 'PHASE TYPE', 'FINAL HES MAPPING',
       'CURRENT LOCATION', 'TOTAL METERS STATUS', 'SUPPLIER',
       'FINAL CATEGORY OK', 'IME AS PER L&T SHIPMENT', 'SIM 19 DISIT', 'IMSI',
       'IP'],
      dtype='object')

In [302]:
# concat the assest_initiation_invent_sb and assest_initiation_invent_nb

NB_SB_COMBINED_DATAFRAME = pd.concat([assest_initiation_invent_sb, assest_initiation_invent_nb], ignore_index=True)

In [303]:
# checking is null

NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME["NETWORK_ID"].isnull()].count()

METERNUMBER                                  0
MFG_SERIAL_NUM                               0
UDC_ID                                       0
NETWORK_ID                                   0
INSTALLATIONNUMBER                           0
STATUS                                       0
INSERT_TIME                                  0
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    0
dtype: int64

In [304]:
# checking duplicate value

NB_SB_COMBINED_DATAFRAME.duplicated().sum()

0

In [305]:
# droping not used column in the NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop(['METERNUMBER', 'INSTALLATIONNUMBER','INSERT_TIME'], axis=1, inplace=True)

In [306]:
NB_SB_COMBINED_DATAFRAME.head()

,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
0,GP4970294,404759210464358,2001:4490:0800:5F25:0000:0000:0000:0001,Meter Shop,SBPDCL
1,GP4962320,404751250451741,2001:4490:0800:A6F5:0000:0000:0000:1,Installed,SBPDCL
2,GP4964119,404751250456722,2001:4490:0800:B5B4:0000:0000:0000:0001,Installed,SBPDCL
3,GP4964992,404751250456883,2001:4490:0800:B655:0000:0000:0000:0001,Installed,SBPDCL
4,GP4964995,404751250456886,2001:4490:0800:B658:0000:0000:0000:0001,Installed,SBPDCL


In [307]:
NB_SB_COMBINED_DATAFRAME.count()

MFG_SERIAL_NUM                               1757842
UDC_ID                                       1757842
NETWORK_ID                                   1757842
STATUS                                       1757842
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    1757842
dtype: int64

In [308]:
# dop duplicate in  NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop_duplicates(subset=['MFG_SERIAL_NUM','UDC_ID',"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], keep='first', inplace=True, ignore_index=True)

In [309]:
NB_SB_COMBINED_DATAFRAME.count()[0]

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_12276\1323936542.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  NB_SB_COMBINED_DATAFRAME.count()[0]


1695091

In [310]:


# Convert all columns to string data type

NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.astype(str)


NB_SB_COMBINED_DATAFRAME.columns = NB_SB_COMBINED_DATAFRAME.columns.str.upper()



# Remove leading and trailing whitespaces from all columns where dtype is 'O'
NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)


In [311]:
NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME['MFG_SERIAL_NUM']=='LT0141611']

,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
6324,LT0141611,405856706290441,2405:0203:0528:CDCB:0000:0000:0000:0010,Installed,SBPDCL
68064,LT0141611,405856706053513,2405:0203:0522:3F90:0000:0000:0000:0010,Installed,SBPDCL


In [312]:
assest_initiation_invent_sb[assest_initiation_invent_sb['UDC_ID']=='405856706290441']

,METERNUMBER,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,INSTALLATIONNUMBER,STATUS,INSERT_TIME,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
6327,LNTLT0141611,LT0141611,405856706290441,2405:0203:0528:CDCB:0000:0000:0000:0010,5001355156,Installed,12/3/2021 1:04:31 PM,SBPDCL
6328,LNTLT0141611,LT0141611,405856706290441,2405:0203:0528:CDCB:0000:0000:0000:0010,52405527217,Installed,12/3/2021 1:04:31 PM,SBPDCL


# JOIN



In [313]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.', 'FINAL HES MAPPING'], right_on=['MFG_SERIAL_NUM', "DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], how='left')


In [314]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.count()

SL. NO.                                      6009
METER NO.                                    6009
PHASE TYPE                                   6009
FINAL HES MAPPING                            6009
CURRENT LOCATION                             6009
TOTAL METERS STATUS                          6009
SUPPLIER                                     6009
FINAL CATEGORY OK                            6009
IME AS PER L&T SHIPMENT                      6009
SIM 19 DISIT                                 6009
IMSI                                         6009
IP                                           6009
MFG_SERIAL_NUM                               5806
UDC_ID                                       5806
NETWORK_ID                                   5806
STATUS                                       5806
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    5806
dtype: int64

In [315]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\new\LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.csv',index=False)

In [316]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['IMSI'], right_on=['UDC_ID'], how='left')

In [317]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

SL. NO.                                      6009
METER NO.                                    6009
PHASE TYPE                                   6009
FINAL HES MAPPING                            6009
CURRENT LOCATION                             6009
TOTAL METERS STATUS                          6009
SUPPLIER                                     6009
FINAL CATEGORY OK                            6009
IME AS PER L&T SHIPMENT                      6009
SIM 19 DISIT                                 6009
IMSI                                         6009
IP                                           6009
MFG_SERIAL_NUM                               6004
UDC_ID                                       6004
NETWORK_ID                                   6004
STATUS                                       6004
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    6004
dtype: int64

# duplicate row while we join with imsi 

In [318]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[~LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].isnull()].count()

In [319]:
# duplicated row ..  bcz in asset initiation file there are 2 installtion no.  with respect to 1 imsi no.

duplicate_check = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].duplicated(keep=False)]

duplicate_check.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\new\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv',index=False)
duplicate_check.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv",index=False)

In [320]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()


SL. NO.                                      6009
METER NO.                                    6009
PHASE TYPE                                   6009
FINAL HES MAPPING                            6009
CURRENT LOCATION                             6009
TOTAL METERS STATUS                          6009
SUPPLIER                                     6009
FINAL CATEGORY OK                            6009
IME AS PER L&T SHIPMENT                      6009
SIM 19 DISIT                                 6009
IMSI                                         6009
IP                                           6009
MFG_SERIAL_NUM                               6004
UDC_ID                                       6004
NETWORK_ID                                   6004
STATUS                                       6004
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    6004
dtype: int64

In [321]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.drop_duplicates(subset=['IMSI'],inplace=True)


In [322]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

SL. NO.                                      5537
METER NO.                                    5537
PHASE TYPE                                   5537
FINAL HES MAPPING                            5537
CURRENT LOCATION                             5537
TOTAL METERS STATUS                          5537
SUPPLIER                                     5537
FINAL CATEGORY OK                            5537
IME AS PER L&T SHIPMENT                      5537
SIM 19 DISIT                                 5537
IMSI                                         5537
IP                                           5537
MFG_SERIAL_NUM                               5532
UDC_ID                                       5532
NETWORK_ID                                   5532
STATUS                                       5532
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    5532
dtype: int64

In [323]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['METER NO.']=='LT0783643']

##### joining both the table "   LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA   "       and            " LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA "

In [324]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [325]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns={"UDC_ID": "METER_MATCH_UDC_ID", "NETWORK_ID": "METER_MATCH_IP", "STATUS": "METER_MATCH_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"METER_MATCH_UTILITY"},
    inplace=True
)



In [326]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column in modified column

In [327]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN

Index(['SL. NO.', 'METER NO.', 'PHASE TYPE', 'FINAL HES MAPPING',
       'CURRENT LOCATION', 'TOTAL METERS STATUS', 'SUPPLIER',
       'FINAL CATEGORY OK', 'IME AS PER L&T SHIPMENT', 'SIM 19 DISIT', 'IMSI',
       'IP', 'MFG_SERIAL_NUM', 'METER_MATCH_UDC_ID', 'METER_MATCH_IP',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY'],
      dtype='object')

In [328]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN]

In [329]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head()

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL
1,2,LT0896844,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065023304,8991856040706440609,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,LT0896844,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL
2,3,LT0057112,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),861089057994155,8991856050706192835,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,LT0057112,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,Installed,NBPDCL
3,4,LT0188615,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),869810051153205,8991856050706347461,405856706347461,2405:0203:0523:7C1A:0000:0000:0000:0010,LT0188615,405856706347461,2405:0203:0523:7C1A:0000:0000:0000:0010,Installed,NBPDCL
4,5,LT0190120,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),869810051552232,8991856050706356047,405856706356047,2405:0203:0523:81FB:0000:0000:0000:0010,LT0190120,405856706356047,2405:0203:0523:81FB:0000:0000:0000:0010,Installed,NBPDCL


In [384]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED['METER_MATCH_UDC_ID']=='nan'].count()

SL. NO.                    224
METER NO.                  224
PHASE TYPE                 224
FINAL HES MAPPING          224
CURRENT LOCATION           224
TOTAL METERS STATUS        224
SUPPLIER                   224
FINAL CATEGORY OK          224
IME AS PER L&T SHIPMENT    224
SIM 19 DISIT               224
IMSI                       224
IP                         224
MFG_SERIAL_NUM             224
METER_MATCH_UDC_ID         224
METER_MATCH_IP             224
METER_MATCH_STATUS         224
METER_MATCH_UTILITY        224
dtype: int64

In [330]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"METER_MATCH_UDC_ID","METER_MATCH_IP","METER_MATCH_STATUS","METER_MATCH_UTILITY"]]

In [331]:
# LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [332]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns = {"MFG_SERIAL_NUM":"IMSI_MATCHED_METER_NO","NETWORK_ID":"IMSI_MATCHED_IP","STATUS":"IMSI_MATCHED_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"IMSI_MATCHED_UTILITY"},
    inplace=True
)

In [333]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.head(1)

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL


In [334]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN=LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN

Index(['SL. NO.', 'METER NO.', 'PHASE TYPE', 'FINAL HES MAPPING',
       'CURRENT LOCATION', 'TOTAL METERS STATUS', 'SUPPLIER',
       'FINAL CATEGORY OK', 'IME AS PER L&T SHIPMENT', 'SIM 19 DISIT', 'IMSI',
       'IP', 'IMSI_MATCHED_METER_NO', 'UDC_ID', 'IMSI_MATCHED_IP',
       'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY'],
      dtype='object')

In [335]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN]

In [336]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL


In [337]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"IMSI_MATCHED_METER_NO","IMSI_MATCHED_IP","IMSI_MATCHED_STATUS","IMSI_MATCHED_UTILITY"]]

### WORK ON LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED  and  LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED

In [338]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL
1,2,LT0896844,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065023304,8991856040706440609,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,LT0896844,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL
2,3,LT0057112,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),861089057994155,8991856050706192835,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,LT0057112,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,Installed,NBPDCL


In [339]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL
1,2,LT0896844,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065023304,8991856040706440609,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,LT0896844,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL
2,3,LT0057112,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),861089057994155,8991856050706192835,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,LT0057112,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,Installed,NBPDCL


In [340]:
#

# data correction

In [341]:
# Convert all columns to string data type

# change into str
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)


# ----------------------------------------------------------------------------------

# change into upper
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()


# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)




In [342]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL


# Merge IMSI  and METER  file

In [343]:
warehouse_meter_imsi_data= pd.DataFrame(columns=[warehouse_Data.columns])

In [344]:
warehouse_meter_imsi_data

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP


In [345]:
#  empty column add to the  warehouse_meter_imsi_data  which has already some empty column of warehouse data

In [346]:
warehouse_meter_imsi_data['METER_MATCH_UDC_ID'] = None
warehouse_meter_imsi_data['METER_MATCH_IP'] = None
warehouse_meter_imsi_data['METER_MATCH_STATUS'] = None
warehouse_meter_imsi_data['METER_MATCH_UTILITY'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_IP'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_STATUS'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_UTILITY'] = None


In [347]:
warehouse_meter_imsi_data

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY


In [348]:
# this column are not same in all the dataframe so that i used the above method

In [349]:
"""
warehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'FINAL HES MAPPING',
       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',
       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',
       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',
       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',
       'IMSI_MATCHED_UTILITY'])
"""

"\nwarehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'FINAL HES MAPPING',\n       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',\n       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',\n       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',\n       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',\n       'IMSI_MATCHED_UTILITY'])\n"

In [350]:
warehouse_meter_imsi_data.head()

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY


In [351]:
warehouse_meter_imsi_data = pd.merge(LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED, LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[['METER NO.', 'IMSI','IMSI_MATCHED_METER_NO','IMSI_MATCHED_IP','IMSI_MATCHED_STATUS','IMSI_MATCHED_UTILITY']], left_on=['METER NO.', 'IMSI'], right_on=['METER NO.', 'IMSI'], how='left')

In [352]:
warehouse_meter_imsi_data.head(2)

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,LT0884520,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL
1,2,LT0896844,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065023304,8991856040706440609,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,LT0896844,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,LT0896844,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL


In [353]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data = warehouse_meter_imsi_data.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data.columns = warehouse_meter_imsi_data.columns.str.upper()

# ----------------------------------------------------------------------------------


# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data = warehouse_meter_imsi_data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



# SIM SET CHECK STATUS

In [354]:
def fun_sim_status_check(row):
    if (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['FINAL HES MAPPING'] == row['METER_MATCH_UTILITY']):
        return "SAME SET IN SAME UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['FINAL HES MAPPING'] == row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN SAME UTILITY"
    elif (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['FINAL HES MAPPING'] != row['METER_MATCH_UTILITY']):
        return "SAME SET IN DIFFERENT UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['FINAL HES MAPPING'] != row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    else:
        return None

# Apply the function to create a new column
warehouse_meter_imsi_data['SIM SET CHECK STATUS'] = warehouse_meter_imsi_data.apply(fun_sim_status_check, axis=1)



# TRUE - FALSE COLUMN

In [355]:
def fun_check_true_false__METER_SIM(row):
    if row['IMSI']==row['METER_MATCH_UDC_ID']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_SIM"] =  warehouse_meter_imsi_data.apply(fun_check_true_false__METER_SIM, axis=1)

In [356]:
def fun_check_true_false_METER_ORG(row):
    if row['METER_MATCH_UTILITY']==row['FINAL HES MAPPING']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_ORG, axis=1)

In [357]:
def fun_check_true_false_SIM_ORG(row):
    if row['IMSI_MATCHED_UTILITY']==row['FINAL HES MAPPING']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["SIM_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_SIM_ORG, axis=1)

In [358]:
def fun_check_true_false_METER_IP(row):
    if row['METER_MATCH_IP']==row['IP']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_IP"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_IP, axis=1)

# SIM SET CHECK STATUS RECHECK COLUMN

In [359]:
def check_set_and_utility(row):
    if row['METER_SIM'] == True and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "SAME SET IN SAME UTILITY"
    elif row['METER_SIM'] == True and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "SAME SET IN DIFFERENT UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "DIFFERENT SET IN SAME UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    elif row['METER_SIM']== False and row['METER_ORG']== True and row['SIM_ORG']== False and row['METER_IP']== False:
        return "REFURBISHED METER WITH NEW IMSI"
    else:
        return None


warehouse_meter_imsi_data["SIM SET CHECK STATUS RECHECK"] =  warehouse_meter_imsi_data.apply(check_set_and_utility, axis=1)


In [360]:
warehouse_meter_imsi_data.columns

Index(['SL. NO.', 'METER NO.', 'PHASE TYPE', 'FINAL HES MAPPING',
       'CURRENT LOCATION', 'TOTAL METERS STATUS', 'SUPPLIER',
       'FINAL CATEGORY OK', 'IME AS PER L&T SHIPMENT', 'SIM 19 DISIT', 'IMSI',
       'IP', 'MFG_SERIAL_NUM', 'METER_MATCH_UDC_ID', 'METER_MATCH_IP',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY', 'IMSI_MATCHED_METER_NO',
       'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY',
       'SIM SET CHECK STATUS', 'METER_SIM', 'METER_ORG', 'SIM_ORG', 'METER_IP',
       'SIM SET CHECK STATUS RECHECK'],
      dtype='object')

# Action taken column

In [361]:

def fun_action_taken_case_I(row):

    # case 1
    
    if (row['METER_MATCH_STATUS'] == "Installed")  & (row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "SET IS IN INSTALLED STAGE"
    elif (row['METER_MATCH_STATUS'] != "Installed")  & (row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "IMSI IS IN INSTALLED STAGE"
    elif (row['METER_MATCH_STATUS'] == "Installed")  & (row['IMSI_MATCHED_STATUS'] != "Installed"):
        return "METER IS IN INSTALLED STAGE"
    
    # case 2 


    # set 1 and row['METER_IP'] == True
    elif (row['METER_SIM'] == True) and ((row['METER_ORG']  == True) and (row["SIM_ORG"] == True)) and (row['METER_IP'] == True):
        return "METER-SIM LINK FILE REQUIRED"
    
    # set 1 and row['METER_IP'] == False 
    elif (row['METER_SIM'] == True) and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) and (row['METER_IP'] == False) :
        return "SIM AND METER LINK FILE REQUIRED"
    
    # set 2  "SAME SET IN DIFFERENT UTILITY"
    elif (row['METER_SIM'] == True) and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "METER,SIM AND METER-SIM LINK FILE REQUIRED"

    #set 3  "DIFFERENT SET IN SAME UTILITY"
    
    elif (row['METER_SIM'] == False) and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    
    # set 4  "DIFFERENT SET IN DIFFERENT UTILITY"
    elif (row['METER_SIM'] == False) and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "METER,SIM AND METER-SIM LINK FILE REQUIRED"
    


    # new cases are added in this to handle imsi null
    
    elif ( row['IMSI_MATCHED_METER_NO'] =='nan') and ( row['IMSI_MATCHED_IP'] =='nan') and ( row['IMSI_MATCHED_STATUS'] =='nan') and (row['METER_ORG'] == True ):
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( row['IMSI_MATCHED_METER_NO'] =='nan' ) and ( row['IMSI_MATCHED_IP'] =='nan') and ( row['IMSI_MATCHED_STATUS'] =='nan') and (row['METER_ORG'] == False ):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( row['IMSI_MATCHED_METER_NO'] =='nan') and ( row['IMSI_MATCHED_IP'] =='nan') and ( row['IMSI_MATCHED_STATUS'] =='nan')  and ( row['METER_MATCH_UDC_ID'] =='nan') and ( row['METER_MATCH_IP'] =='nan') and ( row['METER_MATCH_STATUS'] =='nan'):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    
    

    
    else:
        return " NEED TO CHECK THIS FILE"
    
    
    
warehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply(fun_action_taken_case_I, axis=1)

In [362]:
warehouse_meter_imsi_data[warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO']!='nan']

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,LT0884520,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
1,2,LT0896844,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065023304,8991856040706440609,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,LT0896844,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,LT0896844,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
2,3,LT0057112,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),861089057994155,8991856050706192835,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,LT0057112,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,Installed,NBPDCL,LT0057112,2405:0203:0520:D727:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
3,4,LT0188615,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),869810051153205,8991856050706347461,405856706347461,2405:0203:0523:7C1A:0000:0000:0000:0010,LT0188615,405856706347461,2405:0203:0523:7C1A:0000:0000:0000:0010,Installed,NBPDCL,LT0188615,2405:0203:0523:7C1A:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
4,5,LT0190120,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),869810051552232,8991856050706356047,405856706356047,2405:0203:0523:81FB:0000:0000:0000:0010,LT0190120,405856706356047,2405:0203:0523:81FB:0000:0000:0000:0010,Installed,NBPDCL,LT0190120,2405:0203:0523:81FB:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6004,5353,LT0581847,1 Phase,SBPDCL,NOT MAP IN HES,S2S Repaired,L&T,nan,861089052057958,8991856050706169800,405856706169800,2405:0203:0520:858D:0000:0000:0000:0010,LT0581847,405856706655454,2405:0203:0529:AF11:0000:0000:0000:0010,Retired,SBPDCL,LT0031848,2405:0203:0520:858C:0000:0000:0000:0010,Installed,NBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,IMSI IS IN INSTALLED STAGE
6005,5354,LT0922484,1 Phase,SBPDCL,NOT MAP IN HES,S2S Repaired,L&T,nan,861557061102197,8991856040706015363,405856706560385,2405:0203:0528:8D1F:0000:0000:0000:0010,LT0922484,405856707365240,2405:0203:052F:3315:0000:0000:0000:0010,Retired,SBPDCL,LT0515091,2405:0203:0528:8D1F:0000:0000:0000:0010,Installed,NBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,IMSI IS IN INSTALLED STAGE
6006,5355,LT0944815,1 Phase,SBPDCL,NOT MAP IN HES,S2S Repaired,L&T,nan,865681061689007,8991856050706502246,405856706497246,2405:0203:0526:A8BE:0000:0000:0000:0010,LT0944815,405856707037445,2405:0203:052F:846E:0000:0000:0000:0010,Retired,SBPDCL,LT0432771,2405:0203:0526:A8BE:0000:0000:0000:0010,Refurbished,NBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,NEED TO CHECK THIS FILE
6007,5356,LT0094164,1 Phase,SBPDCL,NOT MAP IN HES,S2S Repaired,L&T,nan,869810058626716,8991856040706150270,405856706695291,2405:0203:0525:E23A:0000:0000:0000:0010,LT0094164,405856706

In [363]:
warehouse_meter_imsi_data.head(2)

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,LT0884520,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
1,2,LT0896844,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065023304,8991856040706440609,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,LT0896844,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,LT0896844,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE


In [364]:
"""

def fun_action_taken_new_case(row):

    if (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == True ):
        return "SIM AND METER SIM LINK REQUIRED"
    elif (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == False) :
        return "METER , SIM AND METER SIM LINK REQUIRED"
        
    elif (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == True ) and (row['METER_MATCH_UDC_ID'].isnull()) &  (row['METER_MATCH_IP'].isnull()) &  (row['METER_MATCH_STATUS'].isnull()):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    else:
        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"


warehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)

"""


'\n\ndef fun_action_taken_new_case(row):\n\n    if (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == True ):\n        return "SIM AND METER SIM LINK REQUIRED"\n    elif (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == False) :\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n        \n    elif (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == True ) and (row[\'METER_MATCH_UDC_ID\'].isnull()) &  (row[\'METER_MATCH_IP\'].isnull()) &  (row[\'METER_MATCH_STATUS\'].isnull()):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    else:\n        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"\n\n\nwarehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply

In [365]:
"""

def fun_action_taken_new_case(row):
    if (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS']) and (row['METER_ORG'] == True ):
        return "SIM AND METER SIM LINK REQUIRED"
    elif (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS']) and (row['METER_ORG'] == False ):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    elif (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS'])  and (not row['METER_MATCH_UDC_ID']) and (not row['METER_MATCH_IP']) and (not row['METER_MATCH_STATUS']):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    else:
        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"

warehouse_meter_imsi_data["ACTION TAKEN"] = warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)


"""


'\n\ndef fun_action_taken_new_case(row):\n    if (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\']) and (row[\'METER_ORG\'] == True ):\n        return "SIM AND METER SIM LINK REQUIRED"\n    elif (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\']) and (row[\'METER_ORG\'] == False ):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    elif (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\'])  and (not row[\'METER_MATCH_UDC_ID\']) and (not row[\'METER_MATCH_IP\']) and (not row[\'METER_MATCH_STATUS\']):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    else:\n        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"\n\nwarehouse_meter_imsi_data["ACTION TAKEN"] = warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)\n\n\n'

# Finally done

In [366]:
warehouse_meter_imsi_data.head(1)

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,LT0884520,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE


# FINAL FILE SAVE AT MULTIPLE LOCATION

In [367]:
# Final file at desktop folder

warehouse_meter_imsi_data.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\new\warehouse_Data_meter_sim_final.csv',index=False)


# Final file in code folder

warehouse_meter_imsi_data.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv",index=False)

In [368]:
warehouse_meter_imsi_data_check=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv")

In [369]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data_check.columns = warehouse_meter_imsi_data_check.columns.str.upper()

# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)

# NEED TO CHECK FILE

In [370]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM = warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN']==' NEED TO CHECK THIS FILE']

In [371]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv",index=False)
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\new\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv',index=False)



In [372]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV = pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv")

In [373]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV.head()

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN


In [374]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'].isnull()]

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN


In [375]:
warehouse_meter_imsi_data_check.head()


,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,LT0884520,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
1,2,LT0896844,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065023304,8991856040706440609,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,LT0896844,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,LT0896844,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
2,3,LT0057112,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),861089057994155,8991856050706192835,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,LT0057112,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,Installed,NBPDCL,LT0057112,2405:0203:0520:D727:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
3,4,LT0188615,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),869810051153205,8991856050706347461,405856706347461,2405:0203:0523:7C1A:0000:0000:0000:0010,LT0188615,405856706347461,2405:0203:0523:7C1A:0000:0000:0000:0010,Installed,NBPDCL,LT0188615,2405:0203:0523:7C1A:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
4,5,LT0190120,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),869810051552232,8991856050706356047,405856706356047,2405:0203:0523:81FB:0000:0000:0000:0010,LT0190120,405856706356047,2405:0203:0523:81FB:0000:0000:0000:0010,Installed,NBPDCL,LT0190120,2405:0203:0523:81FB:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE


In [376]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'] != "NEED TO CHECK THIS FILE"]

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,LT0884520,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
1,2,LT0896844,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065023304,8991856040706440609,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,LT0896844,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,LT0896844,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
2,3,LT0057112,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),861089057994155,8991856050706192835,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,LT0057112,405856706192835,2405:0203:0520:D727:0000:0000:0000:0010,Installed,NBPDCL,LT0057112,2405:0203:0520:D727:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
3,4,LT0188615,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),869810051153205,8991856050706347461,405856706347461,2405:0203:0523:7C1A:0000:0000:0000:0010,LT0188615,405856706347461,2405:0203:0523:7C1A:0000:0000:0000:0010,Installed,NBPDCL,LT0188615,2405:0203:0523:7C1A:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
4,5,LT0190120,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),869810051552232,8991856050706356047,405856706356047,2405:0203:0523:81FB:0000:0000:0000:0010,LT0190120,405856706356047,2405:0203:0523:81FB:0000:0000:0000:0010,Installed,NBPDCL,LT0190120,2405:0203:0523:81FB:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,5352,LT0976464,1 Phase,SBPDCL,NOT MAP IN HES,Refurbished,L&T,nan,869810055869731,8991856050706493714,405856706488714,2405:0203:0525:CB74:0000:0000:0000:0010,LT0976464,405856707649593,2405:0203:0530:B0B9:0000:0000:0000:0010,Retired,SBPDCL,LT0365549,2405:0203:0525:CB74:0000:0000:0000:0010,Installed,NBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,IMSI IS IN INSTALLED STAGE
6004,5353,LT0581847,1 Phase,SBPDCL,NOT MAP IN HES,S2S Repaired,L&T,nan,861089052057958,8991856050706169800,405856706169800,2405:0203:0520:858D:0000:0000:0000:0010,LT0581847,405856706655454,2405:0203:0529:AF11:0000:0000:0000:0010,Retired,SBPDCL,LT0031848,2405:0203:0520:858C:0000:0000:0000:0010,Installed,NBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,IMSI IS IN INSTALLED STAGE
6005,5354,LT0922484,1 Phase,SBPDCL,NOT MAP IN HES,S2S Repaired,L&T,nan,861557061102197,8991856040706015363,405856706560385,2405:0203:0528:8D1F:0000:0000:0000:0010,LT0922484,405856707365240,2405:0203:052F:3315:0000:0000:0000:0010,Retired,SBPDCL,LT0515091,2405:0203:0528:8D1F:0000:0000:0000:0010,Installed,NBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,IMSI IS IN INSTALLED STAGE
6007,5356,LT0094164,1 Phase,SBPDCL,NOT MAP IN HES,S2S Repaired,L&T,nan,869810058626716,8991856040706150270,405856706695291,2405:0203:0525:E23A:0000:0000:0000:0010,LT0094164,405856706

In [377]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['SET IS IN INSTALLED STAGE', 'METER-SIM LINK FILE REQUIRED',
       'IMSI IS IN INSTALLED STAGE', 'METER IS IN INSTALLED STAGE',
       'SIM AND METER-SIM LINK FILE REQUIRED', 'NEED TO CHECK THIS FILE',
       'METER,SIM AND METER-SIM LINK FILE REQUIRED',
       'SIM AND METER LINK FILE REQUIRED'], dtype=object)

In [378]:
warehouse_meter_imsi_data_check.head(2)

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,LT0884520,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065017256,8991856040706586279,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,LT0884520,405856707131288,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,LT0884520,2405:0203:052E:6B10:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE
1,2,LT0896844,1 Phase,NBPDCL,In Warehouse,S2S(PLD),L&T,S2S(PLD),868886065023304,8991856040706440609,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,LT0896844,405856706985618,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,LT0896844,2405:0203:052E:9B35:0000:0000:0000:0010,Installed,NBPDCL,SAME SET IN SAME UTILITY,True,True,True,True,SAME SET IN SAME UTILITY,SET IS IN INSTALLED STAGE


In [379]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['SET IS IN INSTALLED STAGE', 'METER-SIM LINK FILE REQUIRED',
       'IMSI IS IN INSTALLED STAGE', 'METER IS IN INSTALLED STAGE',
       'SIM AND METER-SIM LINK FILE REQUIRED', 'NEED TO CHECK THIS FILE',
       'METER,SIM AND METER-SIM LINK FILE REQUIRED',
       'SIM AND METER LINK FILE REQUIRED'], dtype=object)

In [380]:
warehouse_meter_imsi_data_check['SIM SET CHECK STATUS RECHECK'].unique()

array(['SAME SET IN SAME UTILITY', 'DIFFERENT SET IN SAME UTILITY', 'nan',
       'REFURBISHED METER WITH NEW IMSI',
       'DIFFERENT SET IN DIFFERENT UTILITY'], dtype=object)

In [381]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['IMSI_MATCHED_METER_NO']=="nan"]

,SL. NO.,METER NO.,PHASE TYPE,FINAL HES MAPPING,CURRENT LOCATION,TOTAL METERS STATUS,SUPPLIER,FINAL CATEGORY OK,IME AS PER L&T SHIPMENT,SIM 19 DISIT,IMSI,IP,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
1988,4864,LT0143605,1 Phase,NBPDCL,NOT MAP IN HES,Refurbished,L&T,nan,861190056099168,8991856050706102326,405856706102326,2405:0203:0528:CDE3:0000:0000:0000:0010,LT0143605,nan,nan,Meter Shop,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED
1989,4865,LT0166555,1 Phase,NBPDCL,NOT MAP IN HES,Refurbished,L&T,nan,861190057099704,8991856050706102325,405856706102325,2405:0203:0528:CE0F:0000:0000:0000:0010,LT0166555,nan,nan,Meter Shop,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED
1990,4866,LT0168391,1 Phase,NBPDCL,NOT MAP IN HES,Refurbished,L&T,nan,861190056989004,8991856050706102138,405856706102138,2405:0203:0528:CDDA:0000:0000:0000:0010,LT0168391,nan,nan,Meter Shop,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED
1991,4867,LT0171228,1 Phase,NBPDCL,NOT MAP IN HES,Refurbished,L&T,nan,861190057307818,8991856050706352212,405856706352212,2405:0203:0528:CDE1:0000:0000:0000:0010,LT0171228,nan,nan,Meter Shop,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED
1992,4868,LT0181976,1 Phase,NBPDCL,NOT MAP IN HES,Refurbished,L&T,nan,861190056133736,8991856050706101941,405856706101941,2405:0203:0528:CDD2:0000:0000:0000:0010,LT0181976,nan,nan,Meter Shop,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED


In [382]:
warehouse_meter_imsi_data_check['METER_MATCH_UDC_ID'].dtype

dtype('O')

In [383]:
warehouse_meter_imsi_data_check.count()

SL. NO.                         6009
METER NO.                       6009
PHASE TYPE                      6009
FINAL HES MAPPING               6009
CURRENT LOCATION                6009
TOTAL METERS STATUS             6009
SUPPLIER                        6009
FINAL CATEGORY OK               6009
IME AS PER L&T SHIPMENT         6009
SIM 19 DISIT                    6009
IMSI                            6009
IP                              6009
MFG_SERIAL_NUM                  6009
METER_MATCH_UDC_ID              6009
METER_MATCH_IP                  6009
METER_MATCH_STATUS              6009
METER_MATCH_UTILITY             6009
IMSI_MATCHED_METER_NO           6009
IMSI_MATCHED_IP                 6009
IMSI_MATCHED_STATUS             6009
IMSI_MATCHED_UTILITY            6009
SIM SET CHECK STATUS            6009
METER_SIM                       6009
METER_ORG                       6009
SIM_ORG                         6009
METER_IP                        6009
SIM SET CHECK STATUS RECHECK    6009
A